 # Import Dataset

In [34]:
import pandas as pd
import numpy as np

In [35]:
from cmdstanpy import CmdStanModel
import arviz as az

import matplotlib.pyplot as plt

In [36]:
with open("fecondità_finale", "rb") as f:
    dataset=pd.read_csv(f)

In [37]:
Y=np.array(dataset["tasso.di.fecondità.totale"])

In [38]:
maxY=np.max(Y)
minY=np.min(Y)
Y=(Y-minY)/(maxY-minY)

In [39]:
#X1=np.array(dataset["età.media.delle.madri.al.parto"])
#maxX1=np.max(X1)
#minX1=np.min(X1)
#X1=(X1-minX1)/(maxX1-minX1)
#X2=np.array(dataset["età.media.dei.padri.alla.nascita.del.figlio"])
#maxX2=np.max(X2)
#minX2=np.min(X2)
#X2=(X2-minX2)/(maxX2-minX2)
#X3=np.array(dataset["tasso.di.occupazione.totale"])
#maxX3=np.max(X3)
#minX3=np.min(X3)
#X3=(X3-minX3)/(maxX3-minX3)

In [49]:
#X=np.ones((1060,4))
#X[:,1]=X1
#X[:,2]=X2
#X[:,3]=X3
#X

X=np.ones((1060,2))
X3=np.array(dataset["tasso.di.occupazione.totale"])
X[:,1]=X3

In [50]:
I=106
T=10
H=10
P=1

In [28]:
dataset

,Territorio,TIME,età.media.delle.madri.al.parto,età.media.dei.padri.alla.nascita.del.figlio,tasso.di.fecondità.totale,tasso.di.inattività.dei.maschi,tasso.di.inattività.delle.femmine,tasso.di.inattività.totale,tasso.di.occupazione.dei.maschi,tasso.di.occupazione.delle.femmine,tasso.di.occupazione.totale
0,Torino,2011,31.59,35.00,1.43,40.212630,54.400780,47.626363,54.855120,41.019902,47.625853
1,Torino,2012,31.68,35.05,1.42,40.652276,54.072296,47.665601,54.027223,40.995269,47.216702
2,Torino,2013,31.73,35.09,1.40,41.097223,54.988974,48.350544,52.351428,39.833208,45.815270
3,Torino,2014,31.80,35.17,1.40,40.677300,54.461020,47.867962,51.839929,39.479064,45.391494
4,Torino,2015,31.96,35.32,1.37,39.477634,54.670485,47.403858,53.427950,39.817704,46.327427
...,...,...,...,...,...,...,...,...,...,...,...
1055,Barletta-Andria-Trani,2016,31.80,34.91,1.33,44.206319,74.691900,59.737920,47.229109,18.919779,32.806251
1056,Barletta-Andria-Trani,2017,31.82,35.09,1.31,45.726624,72.622419,59.430846,45.939482,21.605327,33.540231
1057,Barletta-Andria-Trani,2018,31.96,35.17,1.29,46.478675,72.026655,59.496393,47.123252,22.817231,34.738073
1058,Barletta-Andria-Trani,2019,32.09,35.20,1.26,46.497049,73.013733,60.008879,47.245038,21.901471,34.331282


In [42]:
with open("proximity_matrix", "rb") as f:
    W_raw=pd.read_csv(f)

In [43]:
W_raw=np.matrix(W_raw)

# Prepare data for model

In [51]:

mu_0 = .4*np.ones(P+1) #as in the paper
mu_w_1 = np.zeros(I) #as in the paper
sigma_0 = 1 #covariance matrix


data = {
    "I": I,
    "T": T,
    "P": P,
    "H": H,
    "y": Y,
    "X": X,
    "W_raw": W_raw,
    "mu_0": mu_0,
    "sigma_0": sigma_0,
    "mu_w_1": mu_w_1,
    "a_alpha": 3,
    "b_alpha": 2,
    "a_tau2": 3,
    "b_tau2": 2,
    "a_sigma2": 3,
    "b_sigma2": 2,
    "rho": 0.95,
    "a_xi": 1,
    "b_xi": 1
}

# Model

In [48]:
dp_mix_stan = """
data
{
    int I; // number of areal locations
    int T; // number of time steps
    int P; // number of covariates
    int H; // truncation of stick breaking construction dp
    
    vector[I*T]     y; // output values
    matrix[I*T,P+1] X; // covariate matrix
    // syntax: y(i,t) = y[T*(i-1) + t]
    
    matrix[I,I] W_raw; // proximity matrix
      
    // hyperpar vector of regressors
    vector[P+1] mu_0; 
    real        sigma_0;
    
    // w_1
    vector[I] mu_w_1;
    
    // alpha
    real a_alpha;
    real b_alpha;
    
    // tau^2
    real a_tau2;
    real b_tau2;
    
    // sigma^2
    real a_sigma2;
    real b_sigma2;
    
    // rho
    real rho;
    
    //xis
    real a_xi;
    real b_xi;
}

transformed data
{
    vector[I] ones_I;
    for (i in 1:I)
        ones_I[i] = 1;
    
    matrix[I,I] eye_I;
    eye_I = diag_matrix(ones_I);
        
    matrix[I,I] W;
    W = diag_matrix(W_raw*ones_I) - W_raw;
}

parameters
{
    real<lower=0> alpha;
    real<lower=0> sigma2;
    real<lower=0> tau2;    
    
    // autoregressive coefficients still to reparametrize
    real<lower=0,upper=1> xi_constructor;
    
    // random effects
    matrix[T,I]                ws;
    
    // betas for the mixture of the dirichlet process
    matrix[P+1,H]              betas; 
    
    // for the construction of the dirichlet process
    vector<lower=0,upper=1>[H-1] vs;
}

transformed parameters
{   // weights stick breaking construction
    simplex[H] omegas; 
    
    // sbc stuff
    vector[H-1] cumprod_one_mv;    
    cumprod_one_mv = exp(cumulative_sum(log1m(vs)));
    
    omegas[1] = vs[1];
    omegas[2:(H-1)] = vs[2:(H-1)] .* cumprod_one_mv[1:(H-2)];
    omegas[H] = cumprod_one_mv[H-1];

}

model
{
    alpha  ~ gamma(a_alpha,b_alpha);
    sigma2 ~ inv_gamma(a_sigma2,b_sigma2);
    tau2   ~ inv_gamma(a_tau2,b_tau2);
    vs     ~ beta(1,alpha);
    real xi;
    xi_constructor ~ beta(a_xi,b_xi);
    xi=2*xi_constructor-1;
     
    matrix[I,I] inv_Q;
    inv_Q = inverse_spd(rho*W + (1-rho)*eye_I);
    
    ws[1,1:I] ~ multi_normal(mu_w_1, tau2*inv_Q);
    
    for (t in 2:T)
        ws[t,1:I] ~ multi_normal(ws[t-1,1:I]*xi, tau2*inv_Q);
    
    for (h in 1:H)
        betas[1:P+1,h] ~ normal(mu_0, sigma_0);
        
    for (i in 1:I) {
        vector[H] log_probs;
        
        for (h in 1:H) 
            log_probs[h] = log(omegas[h]) + 
            normal_lpdf(y[T*(i-1)+1:i*T] | X[T*(i-1)+1:i*T, 1:P+1]*betas[1:P+1,h] + ws[1:T,i], sigma2);
        
        target += log_sum_exp(log_probs);
    }
    matrix[I,H] log_probs;
    for (i in 1:I) 
    {
        for (h in 1:H) 
            log_probs[i,h] = log(omegas[h]) + 
            normal_lpdf(y[T*(i-1)+1:i*T] | X[T*(i-1)+1:i*T, 1:P+1]*betas[1:P+1,h] + ws[1:T,i], sigma2);
    
    }
}

generated quantities 
{   
    // vector of cluster allocations
    vector[I] s;
    
    matrix[I,H] log_probs;
    for (i in 1:I) 
    {
        for (h in 1:H) 
            log_probs[i,h] = log(omegas[h]) + 
            normal_lpdf(y[T*(i-1)+1:i*T] | X[T*(i-1)+1:i*T, 1:P+1]*betas[1:P+1,h] + ws[1:T,i], sigma2);
    
    }
    for (i in 1:I)
        s[i] = categorical_rng(softmax(log_probs[i,1:H]'));

    
}
"""

stan_file = "True_Dataset.stan"

with open(stan_file, "w") as fp:
    fp.write(dp_mix_stan)
    
dp_mix = CmdStanModel(stan_file=stan_file)

10:52:06 - cmdstanpy - INFO - compiling stan file /home/anna-ubuntu/bayesian-project/True_Dataset.stan to exe file /home/anna-ubuntu/bayesian-project/True_Dataset
10:53:57 - cmdstanpy - INFO - compiled model executable: /home/anna-ubuntu/bayesian-project/True_Dataset


In [53]:
fit = dp_mix.sample(data
                    , iter_warmup = 4000
                    , iter_sampling = 4000
#                    , adapt_init_phase = 1500
#                    , adapt_metric_window = 1500
#                   , adapt_step_size = 1500
                     , show_console = True
                     , chains=4
#                     , adapt_delta=0.9
#                    , step_size = 0.2
#                     , max_treedepth = 100
                   )

12:10:33 - cmdstanpy - INFO - Chain [1] start processing
12:10:33 - cmdstanpy - INFO - Chain [2] start processing
12:10:33 - cmdstanpy - INFO - Chain [3] start processing
12:10:33 - cmdstanpy - INFO - Chain [4] start processing


Chain [1] method = sample (Default)
Chain [1] sample
Chain [1] num_samples = 4000
Chain [1] num_warmup = 4000
Chain [1] save_warmup = 0 (Default)
Chain [1] thin = 1 (Default)
Chain [1] adapt
Chain [1] engaged = 1 (Default)
Chain [1] gamma = 0.050000000000000003 (Default)
Chain [1] delta = 0.80000000000000004 (Default)
Chain [1] kappa = 0.75 (Default)
Chain [1] t0 = 10 (Default)
Chain [1] init_buffer = 75 (Default)
Chain [1] term_buffer = 50 (Default)
Chain [1] window = 25 (Default)
Chain [1] algorithm = hmc (Default)
Chain [1] hmc
Chain [1] engine = nuts (Default)
Chain [1] nuts
Chain [1] max_depth = 10 (Default)
Chain [1] metric = diag_e (Default)
Chain [1] metric_file =  (Default)
Chain [1] stepsize = 1 (Default)
Chain [1] stepsize_jitter = 0 (Default)
Chain [1] num_chains = 1 (Default)
Chain [1] id = 1 (Default)
Chain [1] data
Chain [1] file = /tmp/tmp888p5vx9/yr2nm7fs.json
Chain [1] init = 2 (Default)
Chain [1] random
Chain [1] seed = 41060
Chain [1] output
Chain [1] file = /tmp/tm

Chain [1] Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Chain [1] Exception: multi_normal_lpdf: Covariance matrix is not symmetric. Covariance matrix[1,21] = 5.62999e+139, but Covariance matrix[21,1] = 5.62999e+139 (in '/home/anna-ubuntu/bayesian-project/True_Dataset.stan', line 101, column 4 to column 49)
Chain [1] If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
Chain [1] but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
Chain [1] 
Chain [2] Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Chain [2] Exception: multi_normal_lpdf: Covariance matrix is not symmetric. Covariance matrix[1,2] = -nan, but Covariance matrix[2,1] = -nan (in '/home/anna-ubuntu/bayesian-project/True_Dataset.stan', line 101, column 4 to column 49)
C

Chain [4] Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Chain [4] Exception: multi_normal_lpdf: Covariance matrix is not symmetric. Covariance matrix[1,21] = 1.06811e+177, but Covariance matrix[21,1] = 1.06811e+177 (in '/home/anna-ubuntu/bayesian-project/True_Dataset.stan', line 101, column 4 to column 49)
Chain [4] If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
Chain [4] but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
Chain [4] 
Chain [4] Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Chain [4] Exception: multi_normal_lpdf: Covariance matrix is not symmetric. Covariance matrix[1,21] = 1.50993e+44, but Covariance matrix[21,1] = 1.50993e+44 (in '/home/anna-ubuntu/bayesian-project/True_Dataset.stan', line 101, column 4

12:17:56 - cmdstanpy - INFO - Chain [4] done processing
12:17:56 - cmdstanpy - ERROR - Chain [4] error: terminated by signal 2 Unknown error -2
12:17:56 - cmdstanpy - INFO - Chain [2] done processing
12:17:56 - cmdstanpy - INFO - Chain [3] done processing
12:17:56 - cmdstanpy - INFO - Chain [1] done processing
12:17:57 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 2 Unknown error -2
12:17:57 - cmdstanpy - ERROR - Chain [3] error: terminated by signal 2 Unknown error -2
12:17:56 - cmdstanpy - ERROR - Chain [2] error: terminated by signal 2 Unknown error -2


Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 
Chain [1] 

KeyboardInterrupt: 

# Model Saving

In [ ]:
s = (fit.s).astype(int)

np.savetxt("true_dataset_posterior_s.csv", s, delimiter=",")
s.shape

In [ ]:
np.savetxt("true_dataset_posterior_betas.csv", fit.betas, delimiter=",")
fit.betas.shape

In [ ]:
np.savetxt("true_dataset_posterior_xi.csv", fit.xi, delimiter=",")
np.savetxt("true_dataset_posterior_alpha.csv", fit.alpha, delimiter=",")

# Plots

In [ ]:
fit_az = az.from_cmdstanpy(fit)
az.plot_trace(fit_az,var_names=["betas"], compact=False)

In [ ]:
fit_az = az.from_cmdstanpy(fit)
az.plot_posterior(fit_az,var_names=["betas"])